# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#!pip install git+http://github.com/scikit-learn/scikit-learn.git

In [2]:
# import libraries
import warnings
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))
    
import pickle
    
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davidhartmann/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davidhartmann/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/davidhartmann/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davidhartmann/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
def load_data(path_to_database):
    """ Load data from Sqlite database into DataFrame
        INPUTS:
        ------------
        path_to_database - path to Sqlite database
        
        OUTPUTS:
        ------------
        df - pandas DataFrame of database
    """
    engine = create_engine(path_to_database)
    df = pd.read_sql("SELECT * FROM disaster", engine)
    
    # X and Y 
    colnames = df.columns.tolist()
    target_colnames = colnames[4:]

    X = df.message.values
    y = df[target_colnames]
    
    return df, colnames, target_colnames, X, y

df, colnames, target_colnames, X, y = load_data('sqlite:///disaster.db')

# Show DataFrame result, shapes, colnames, target
display(df.head())
printmd('#### Shape of df: ' + str(df.shape))
display(y)
printmd('#### Shape of y: ' + str(y.shape))
printmd('#### colnames')
print(colnames)
printmd('#### target_colnames')
print(target_colnames)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Shape of df: (26028, 40)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Shape of y: (26028, 36)

#### colnames

['id', 'message', 'original', 'genre', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


#### target_colnames

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    """ function that will 
        - replace urls via placeholder,
        - tokenize words, 
        - Remove stop words,
        - Lemmatize, normalize all words to lower case, remove white spaces 
         
        INPUTS:
        ------------
        text - text as string
        
        OUTPUTS:
        ------------
        clean_tokens - a list of cleaned words
    
    """
    # Detect URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]  
    
    # Lemmatize, normalize all words to lower case, remove white spaces 
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

## Create a pipeline design
In order to successfully predict classification results for the 36 categories a pipeline helps to organize data preporcessing and data flow based on ETL and NLP.
The used pipeline is structured in the following way:
1. Create a CountVectorizer object to create a Bag of Words
2. 


In [6]:
def build_model(X_train, y_train, pipeline_name='pipeline_1'):
    """ Build a ML pipeline
        INPUTS:
        ------------
        pipeline_name - string name for calling a specific pipeline
        X_train - input features for training 
        y_train - target values
        
        OUTPUTS:
        ------------
        pipeline - sklearn pipeline object which is ETL and NLP 'pipelined', i.e. 
        - tokenization (see tokenize() method), 
        - Transformer method: vecorization via CountVectorizer 
        - Transformer method: tfidf (matrix of weights product, i.e. term frequency times inverse document frequency)
        - Predictor method: MultiOutputClassifier(RandomForestClassifier()) - used to get predictions for each category
        
    """
    if pipeline_name == 'pipeline_1':
        print('pipeline_1 chosen')
        pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier())),
                #('clf', SGDClassifier()),
            ])
        return pipeline
       
    
    if pipeline_name == 'pipeline_2':
        print('pipeline_2 chosen')
        pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                #('clf', SGDClassifier()),
            ])
        
        # uncommenting more parameters will give better exploring power but will
        # increase processing time in a combinatorial way
        parameters = {
            'vect__max_df': (0.5, 0.75, 1.0),
            # 'vect__max_features': (None, 5000, 10000, 50000),
            'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
            # 'tfidf__use_idf': (True, False),
            # 'tfidf__norm': ('l1', 'l2'),
            'clf__estimator__min_samples_split': [2, 4],
            #'clf__estimator__max_features': ['log2', 'auto', 'sqrt', None],
            #'clf__estimator__criterion': ['gini', 'entropy'],
            #'clf__estimator__max_depth': [None, 25, 50, 100, 150, 200],

            # FOR SGDClassifier():
            #'clf__max_iter': (20,),
            #'clf__alpha': (0.00001, 0.000001),
            #'clf__penalty': ('l2', 'elasticnet'),
            # 'clf__max_iter': (10, 50, 80),
        }
        cv = GridSearchCV(estimator=pipeline, param_grid=parameters)
        cv.fit(X_train,y_train)
        return cv

        
    if pipeline_name == 'pipeline_3':
        print('pipeline_3 chosen')
        pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

        parameters = {
            'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
            'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
            'features__text_pipeline__vect__max_features': (None, 5000, 10000),
            'features__text_pipeline__tfidf__use_idf': (True, False),
            'clf__estimator__n_estimators': [50, 100, 200],
            'clf__estimator__min_samples_split': [2, 3, 4],
            'features__transformer_weights': (
                {'text_pipeline': 1, 'starting_verb': 0.5},
                {'text_pipeline': 0.5, 'starting_verb': 1},
                {'text_pipeline': 0.8, 'starting_verb': 1},
            )
        }
        
        cv = GridSearchCV(estimator=pipeline, param_grid=parameters)
        cv.fit(X_train,y_train)
        return cv

In [7]:
def display_results(model, y_test, y_pred):
    """ Get a classification report from testing results including precision, recall, f1-score an accuracy 
        INPUTS:
        ------------
        model - the trained model based on the actual pipeline
        y_test - DataFrame of true target values 
        y_pred - numpy arrays of predicted targets
        
        OUTPUTS:
        ------------
        print statements for precision, recall, f1-score an accuracy for each class
    """
    
    #print(classification_report(y_test, y_pred, target_names=y_test.keys()))
    accuracy = (y_pred == y_test).mean()
    df_classification_report = pd.DataFrame(classification_report(y_test, y_pred, target_names=y_test.keys(),  output_dict=True))
    df_classification_report = pd.concat([df_classification_report.T, accuracy], axis=1).reindex(df_classification_report.T.index)
    df_classification_report.columns = ['f1_score', 'precision', 'recall', 'support', 'accuracy']
    display(df_classification_report)
    print('')
    printmd('#### Total accuracy = ' + str(round(accuracy.mean(),2)))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
def train_predict(pipeline_name):
    df, colnames, target_colnames, X, y = load_data('sqlite:///disaster.db')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = build_model(X_train, y_train, pipeline_name)
    print('Training started...')
    model.fit(X_train, y_train)
    print('... training finished')
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    
    return model, X_train, X_test, y_train, y_test, y_pred

In [9]:
def save_model(model, path):
    """ Saves the model
        
        INPUTS:
        ------------
        model: model to be saved
        path: model filepath 
        
        OUTPUTS:
        ------------
        save model as a pickle file
    """  
    filename = path
    pickle.dump(model, open(filename, 'wb'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
model_1, X_train, X_test, y_train, y_test, y_pred = train_predict(pipeline_name='pipeline_1')
save_model(model_1, 'finalized_model_1.sav')

pipeline_1 chosen
Training started...


In [ ]:
print(type(y_test))
print(type(y_pred))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# predict on test data
model_2, X_train, X_test, y_train, y_test, y_pred = train_predict(pipeline_name='pipeline_2')
save_model(model_2, 'finalized_model_2.sav')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# predict on test data
model_3, X_train, X_test, y_train, y_test, y_pred = train_predict(pipeline_name='pipeline_3')
save_model(model_3, 'finalized_model_3.sav')

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, path):
    """ Saves the model
        INPUTS:
        ------------
        model: The model to be saved
        path: model Filepath 
        
        OUTPUTS:
        ------------
        save the model as a pickle file
    """  
    filename = 'finalized_model.sav'
    pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.